<a href="https://colab.research.google.com/github/componavt/neural_synset/blob/master/src/dataset/wikt_labels_tokenizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading labels with definitions

Source code: [Loading a custom dataset](https://colab.research.google.com/github/huggingface/notebooks/blob/master/course/videos/load_custom_dataset.ipynb#scrollTo=D2ekPOyykZDq), [video](https://www.youtube.com/watch?v=HyQgpJTkRdE).

Video: [The pipeline function](https://www.youtube.com/watch?v=tiZFewofSLM).

Install the Transformers and Datasets libraries to run this notebook.

In [ ]:
! pip install -U accelerate
! pip install -U transformers

! pip install datasets transformers[sentencepiece]
! pip install torch               # required by TrainingArguments
! pip install transformers[torch] # required by TrainingArguments

In [ ]:
!wget https://github.com/componavt/neural_synset/raw/master/data/label_meaning.csv

In [4]:
cat label_meaning.csv

"word"|"meaning"|"книжн."|"ирон."|"религ."|"груб."
подвизаться|осуществлять деятельность, работать, действовать в какой-нибудь области|1|1|0|0
подвизаться|совершать подвиг в чём-либо, часто о ежедневном борении|0|0|1|0
заткнуться|то же, что замолчать; перестать говорить, кричать, плакать; замолкнуть|0|0|0|1
пустобрёх|тот, кто говорит много пустого и несерьёзного; болтун|0|0|0|1
излаять|сильно изругать|0|0|0|1
бизнес-дама|о предпринимательнице|0|1|0|0
агнец божий|кроткий, робкий, безобидный человек|0|1|0|0
всезнайка|человек, который считает себя знающим всё|0|1|0|0
галантерейный|относящийся к галантерее|0|0|0|0
галантерейный|чрезмерно любезный, вежливый до слащавости|0|1|0|0
дитятя|дитя, ребёнок, чадо|0|1|0|0


In [5]:
from datasets import load_dataset

ds = load_dataset("csv", data_files="label_meaning.csv", sep="|")
#ds = load_dataset("csv", data_files="label_meaning.csv", sep="|", split='train')
#ds["train"]
ds

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['word', 'meaning', 'книжн.', 'ирон.', 'религ.', 'груб.'],
        num_rows: 11
    })
})

In [6]:
# 80% train, 20% test + validation
#da = ds.train_test_split(test_size=0.2, shuffle=True)
#da
#datushka["train"]
#datushka["test"]

In [7]:
#print(datushka["train"][0])
#print(len(list(datushka["train"])))

# Pipeline: zero shot classification with labels

When more than one label is passed, we assume that there is only one true label and that the others are false so that the output probabilities add up to 1. This can be changed by passing `multi_class=True`:
nlp(sequence_to_classify, candidate_labels, multi_class=True)

Source: huggingface/transformers/[Zero shot classification pipeline #5760 ](https://github.com/huggingface/transformers/pull/5760).

In [8]:
#from transformers import pipeline
#model_name = "MoritzLaurer/mDeBERTa-v3-base-mnli-xnli"
#pipe = pipeline("zero-shot-classification", model=model_name)

In [9]:
#def meaning_iterator():
#    for i in range(0, len(da["train"]), 1):
#        yield da["train"][i]["meaning"]

#print(len(datushka["train"]))
#datushka["train"][0]["meaning"]
#nlp(datushka["train"][0]["meaning"], ["positive", "negative"], multi_label=True)
###pipe(meaning_iterator(), ["positive", "negative"], multi_label=True)

In [10]:
#sequence_to_classify = "тот, кто говорит много пустого и несерьёзного; болтун"
#candidate_labels = ["книжн.", "ирон.", "религ.", "груб."]
#pipe(sequence_to_classify, candidate_labels, multi_label=True)

#pipe(meaning_iterator(), candidate_labels, multi_label=True)
#pipe(da["train"][0]["meaning"], candidate_labels, multi_label=True)

# AutoTokenizer and PyTorch optimized training loop
From [quicktour.ipynb#AutoTokenizer](https://colab.research.google.com/github/huggingface/notebooks/blob/main/transformers_doc/en/quicktour.ipynb#scrollTo=c-mB_1hXw57y&line=1&uniqifier=1)

In [11]:
# setup hyperparameters (learning rate, batch size, and the number of epochs to train for)
from transformers import TrainingArguments
training_args = TrainingArguments(
    output_dir="./pt_training",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    )

In [12]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "MoritzLaurer/mDeBERTa-v3-base-mnli-xnli"
pt_model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.07k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/286 [00:00<?, ?B/s]

In [13]:
encoding = tokenizer( da["train"][0]["meaning"] )
print(encoding)

NameError: name 'da' is not defined

In [ ]:
def tokenize_dataset(dataset):
  return tokenizer(dataset["meaning"])

In [ ]:
#da["train"]
ds

In [ ]:
#dataset = ds["train"].map(tokenize_dataset, batched=True)
dataset = ds.map(tokenize_dataset, batched=True)
dataset

In [ ]:
# create a batch of examples from dataset
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
data_collator

In [ ]:
# 80% train, 20% test + validation
dataset = dataset['train'].train_test_split(test_size=0.2, shuffle=True)
dataset

In [ ]:
# gather all these classes in Trainer:
from transformers import Trainer
trainer = Trainer(
    model=pt_model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)  # doctest: +SKIP

In [ ]:
trainer.train()

#see https://discuss.huggingface.co/t/the-model-did-not-return-a-loss-from-the-inputs-only-the-following-keys-logits-for-reference-the-inputs-it-received-are-input-values/25420